#### BERT 본체는 freeze, 마지막 k개 레이어는 freeze 풀고, classifier head 만 학습

✅ 라이브러리 & 환경 설정

In [1]:
import torch, random
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_scheduler,
)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("✅ Device:", device)

✅ Device: mps


✅ 데이터셋 로드 & 8:1:1 분할

In [2]:
# SST-2 원본 로드
dataset = load_dataset("glue", "sst2")
train_data = dataset["train"]

# 데이터 변환
train_data = train_data.to_pandas()

# 8:1:1 분할
train_split, temp_split = train_test_split(train_data, test_size=0.2, random_state=42)
val_split, test_split = train_test_split(temp_split, test_size=0.5, random_state=42)

# Hugging Face Dataset 형태로 변환
train_dataset = Dataset.from_dict(train_split)
val_dataset   = Dataset.from_dict(val_split)
test_dataset  = Dataset.from_dict(test_split)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

Train: 53879, Val: 6735, Test: 6735


✅ 토크나이저 및 전처리

In [3]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 전처리 함수
def preprocess(batch):
    return tokenizer(batch["sentence"], truncation=True, padding="max_length", max_length=128)

# 모든 split에 적용
datasets_encoded = {
    "train": train_dataset.map(preprocess, batched=True),
    "validation": val_dataset.map(preprocess, batched=True),
    "test": test_dataset.map(preprocess, batched=True), 
}

# 데이터셋 텐서 형식 설정
for name in datasets_encoded:
    datasets_encoded[name] = datasets_encoded[name].rename_column("label", "labels")
    datasets_encoded[name].set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"]
    )

/opt/anaconda3/envs/bert-project/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/53879 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

✅ 모델 구성 (Encoder Freeze)

In [4]:
def init_model(dropout: float, k_layers: int = 4):
    """
    BERT의 마지막 k개 encoder layer + classifier만 학습하는 partial fine-tuning용 초기화 함수.
    ex) k_layers=4 → encoder.layer.8~11 + classifier 학습
    """
    model = AutoModelForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    )

    # 🔹 dropout 조정 (있으면)
    if hasattr(model, "dropout"):
        model.dropout.p = dropout

    # 1) 전체 freeze
    for p in model.parameters():
        p.requires_grad = False

    # 2) 마지막 k개 encoder layer만 requires_grad=True
    total_layers = 12  # BERT-base는 encoder.layer.0~11
    start_layer = total_layers - k_layers

    for name, p in model.named_parameters():
        # encoder.layer.{start_layer~11}.xxx 에 해당하면 학습
        for i in range(start_layer, total_layers):
            tag = f"encoder.layer.{i}."
            if tag in name:
                p.requires_grad = True

        # classifier는 항상 학습
        if name.startswith("classifier."):
            p.requires_grad = True

    model.to(device)

    # 체크용 출력
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"🧩 Partial FT (last {k_layers} layers) trainable params: "
          f"{trainable:,} / {total:,} ({trainable/total*100:.4f}%)")

    return model

✅ Optimizer & Loss 구성

In [5]:
def configure_optimizer(model, lr, num_epochs, train_loader):

    # 🔹 requires_grad=True 인 파라미터만 선택
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    
    optimizer = torch.optim.AdamW(trainable_params, lr=lr)
    
    # 학습률 스케줄러 설정 
    num_training_steps = num_epochs * len(train_loader)
    scheduler = get_scheduler("linear", optimizer=optimizer,
                              num_warmup_steps=0, num_training_steps=num_training_steps)
    # 손실 함수 설정
    criterion = torch.nn.CrossEntropyLoss()
    
    return optimizer, scheduler, criterion

✅ 학습 루프

In [6]:
def train_model(model, optimizer, scheduler, train_loader, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # forward
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.4f}")

✅ 평가 함수

In [7]:
def evaluate(model, data_loader):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # 🔹 예측
            logits = model(**batch).logits
            pred = logits.argmax(-1)

            preds.extend(pred.cpu().numpy())
            labels.extend(batch["labels"].cpu().numpy())

    # 🔹 주요 지표 계산
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    rec = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    print(f"📊 Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")
    return {
        "acc": acc,
        "prec": prec,
        "rec": rec,
        "f1": f1
    }

✅ Random Search 설정

In [8]:
search_space = {
    "lr": [2e-5, 3e-5],
    "batch_size": [16, 32],
    "dropout": [0.1, 0.2],
    "epochs": [3, 4],
}

n_trials = 5  # 전체 실험 횟수

✅ Random Search Training

In [9]:
results = []
best_val_acc = 0.0
best_model = None
best_params = None

k_candidates = [2, 4]   # 마지막 몇 개 레이어를 풀지

for k_layers in k_candidates:
    print("\n" + "="*40)
    print(f"🔹 Partial Fine-Tuning: last {k_layers} layers")
    print("="*40)

    for trial in range(n_trials):
        # 🔹 랜덤 하이퍼파라미터 선택
        params = {k: random.choice(v) for k, v in search_space.items()}
        print(f"\n🎯 [k={k_layers}] Trial {trial+1}/{n_trials} | {params}")

        # 🔹 모델 초기화 (BERT k개 레이어 + head만 학습)
        model = init_model(params["dropout"], k_layers=k_layers)
        train_loader = DataLoader(
            datasets_encoded["train"],
            batch_size=params["batch_size"],
            shuffle=True
        )
        val_loader   = DataLoader(datasets_encoded["validation"], batch_size=64)
        test_loader  = DataLoader(datasets_encoded["test"], batch_size=64)

        # 🔹 Optimizer & Scheduler 구성
        optimizer, scheduler, criterion = configure_optimizer(
            model, params["lr"], params["epochs"], train_loader
        )

        # 🔹 학습 수행
        train_model(model, optimizer, scheduler, train_loader, num_epochs=params["epochs"])

        # 🔹 Validation / Test 평가
        val_metrics  = evaluate(model, val_loader)
        test_metrics = evaluate(model, test_loader)
        print(f"✅ [k={k_layers}] Val: {val_metrics['acc']*100:.2f}% | "
              f"Test: {test_metrics['acc']*100:.2f}%")

        # 🔹 결과 저장
        results.append({
            "k_layers": k_layers,
            "trial": trial + 1,
            "lr": params["lr"],
            "batch_size": params["batch_size"],
            "dropout": params["dropout"],
            "epochs": params["epochs"],
            "val_acc": val_metrics["acc"],
            "val_prec": val_metrics["prec"],
            "val_rec": val_metrics["rec"],
            "val_f1": val_metrics["f1"],
            "test_acc": test_metrics["acc"],
            "test_prec": test_metrics["prec"],
            "test_rec": test_metrics["rec"],
            "test_f1": test_metrics["f1"]
        })

        # 🔹 Best 모델 업데이트 & 저장 (Val 기준)
        if val_metrics['acc'] > best_val_acc:
            best_val_acc = val_metrics['acc']
            best_model   = model
            best_params  = {
                "k_layers": k_layers,
                **params
            }

            model.save_pretrained("./partial_ft_best_model")
            tokenizer.save_pretrained("./partial_ft_best_model")

            print(f"💾 [k={k_layers}] Best model updated! (Val: {best_val_acc*100:.2f}%)")


🔹 Partial Fine-Tuning: last 2 layers

🎯 [k=2] Trial 1/5 | {'lr': 3e-05, 'batch_size': 16, 'dropout': 0.1, 'epochs': 3}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🧩 Partial FT (last 2 layers) trainable params: 14,177,282 / 109,483,778 (12.9492%)


Epoch 1/3: 100%|██████████| 3368/3368 [13:16<00:00,  4.23it/s]


Epoch 1 | Avg Loss: 0.3183


...

Epoch 1 | Avg Loss: 0.2934


Epoch 2/3: 100%|██████████| 1684/1684 [13:55<00:00,  2.02it/s]


Epoch 2 | Avg Loss: 0.2099


Epoch 3/3: 100%|██████████| 1684/1684 [13:56<00:00,  2.01it/s]


Epoch 3 | Avg Loss: 0.1645
📊 Acc: 0.9351 | Prec: 0.9473 | Rec: 0.9365 | F1: 0.9418
📊 Acc: 0.9372 | Prec: 0.9421 | Rec: 0.9463 | F1: 0.9442
✅ [k=4] Val: 93.51% | Test: 93.72%


📊 결과 정리 및 저장

In [10]:
df = pd.DataFrame(results)
df.to_csv("partial_ft_random_search_tvt.csv", index=False)

🔹 최종 결과 요약


In [11]:
print("\n📊 Random Search Summary (Sorted by Validation Accuracy):")
print(df.sort_values("val_acc", ascending=False))
print("\n🏆 Best Trial Parameters:")
print(best_params, f"Val Acc: {best_val_acc*100:.2f}%")


📊 Random Search Summary (Sorted by Validation Accuracy):
   k_layers  trial       lr  batch_size  dropout  epochs   val_acc  val_prec  \
5         4      1  0.00003          16      0.2       3  0.941648  0.952661   
8         4      4  0.00002          16      0.1       4  0.941500  0.953863   
7         4      3  0.00003          16      0.1       4  0.941054  0.951885   
9         4      5  0.00003          32      0.2       3  0.935115  0.947256   
6         4      2  0.00002          32      0.1       4  0.934670  0.947213   
2         2      3  0.00003          32      0.2       4  0.920119  0.935016   
3         2      4  0.00003          32      0.2       4  0.918337  0.939064   
0         2      1  0.00003          16      0.1       3  0.917446  0.933297   
1         2      2  0.00003          32      0.2       3  0.913140  0.935370   
4         2      5  0.00002          32      0.1       4  0.907944  0.925606   

    val_rec    val_f1  test_acc  test_prec  test_rec   test_f

In [12]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,}/{total:,} ({100*trainable/total:.6f}%)")

Trainable: 28,353,026/109,483,778 (25.897011%)


In [13]:
trainable_params = [p for p in model.parameters() if p.requires_grad]
print(f"Optimizer params: {sum(p.numel() for p in trainable_params):,}")

Optimizer params: 28,353,026


In [14]:
for name, param in model.named_parameters():
    if param.grad is not None:
        print(name, param.grad.abs().mean().item())

In [15]:
print(best_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

best modle torch로 저장

In [16]:
torch.save(best_model.state_dict(), "partial_ft.pt")
print("💾 Saved best model weights as partial_ft.pt")

💾 Saved best model weights as partial_ft.pt
